<a href="https://colab.research.google.com/github/JuyongLee1/tmlab_github/blob/main/0910_%ED%8A%B9%ED%97%88%EC%B6%94%EC%B6%9C%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 패키지 가져오기
import pandas as pd
import requests
import progressbar
import time
import os
from os.path import join
from bs4 import BeautifulSoup
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
script_path = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# search-gp.csv 파일을 데이터프레임으로 불러오기
search_df = pd.read_csv(join(script_path, 'gp-search_0910.csv'), skiprows=[0])


In [ ]:
# 이 부분은 코드에 재개 기능을 추가합니다. 이전 실행의 결과를 코드 경로에서 불러와 결과의 마지막 인덱스에서부터 search-gp.csv를 슬라이싱합니다.
if os.path.isfile(join(script_path, 'patents_data_0910.csv')):
    result = pd.read_csv(join(script_path, 'patents_data_0910.csv'), index_col=0)
    search_df = search_df.loc[result.index[-1] + 1:, :]
else:
    result = pd.DataFrame(columns=['ID', 'Title', 'Abstract', 'Description', 'Claims', 'Inventors', 'Current Assignee', 'Patent Office', 'Publication Date', 'URL'])

# 스크랩되지 않은 링크 목록을 불러옵니다.
if os.path.isfile(join(script_path, 'not_scrap_pickle_0910')):
    with open(join(script_path, 'not_scrap_pickle_0910'), 'rb') as fp:
        not_scraped = pickle.load(fp)
else:
    not_scraped = []

In [ ]:
# 각 요청에 대해 Google 서버로 전송할 사용자 에이전트 설정
h = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}
# h = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [ ]:
# search-gp.csv의 각 행을 반복하면서 서버에 요청을 보냅니다.
for (index, row), i in zip(search_df.iterrows(), progressbar.progressbar(range(len(search_df)))):
    link = row['result link']
    # Google Patents로 요청을 보내고 특허 페이지의 소스를 스크랩합니다.
    try:
        r = requests.get(link, headers=h)
    except requests.exceptions.ConnectionError as e:
        not_scraped.append(link)
        print(e, '\n\n')
        # 에러 비율이 20% 이상인 경우 프로그램을 종료합니다.
        if len(not_scraped) / int(index) >= 0.2:
            print('\n요청 중 절반 이상이 에러로 인해 실패했습니다. 이유를 확인하려면 출력 내용을 읽어보세요.\n')
            break
        continue
    # BeautifulSoup을 사용하여 HTML에서 정보 추출
    bs = BeautifulSoup(r.content, 'html.parser')

    abst = bs.find('section', {'itemprop': 'abstract'})
    # 요약이 없는 경우 처리
    if abst is not None:
        # 요약에 비영어 문단이 있는 경우 처리
        if abst.find('span', class_='notranslate') is None:
            abst = abst.text.strip()
        else:
            notranslate = [tag.find(class_='google-src-text') for tag in abst.find_all('span', class_='notranslate')]
            for tag in notranslate:
                tag.extract()
            abst = abst.text.strip()
    else:
        abst = 'Not Found'

    # 클레임 섹션 찾기
    claims = bs.find('section', {'itemprop': 'claims'})
    # 클레임이 없는 경우 처리
    if claims is not None:
        # 클레임에 비영어 문단이 있는 경우 처리
        if claims.find('span', class_='notranslate') is None:
            claims = claims.text.strip()
        else:
            notranslate = [tag.find(class_='google-src-text') for tag in claims.find_all('span', class_='notranslate')]
            for tag in notranslate:
                tag.extract()
            claims = claims.text.strip()
    else:
        claims = 'Not Found'

     # 결과 데이터프레임에 정보 추가
    result.at[index, 'ID'] = search_df.at[index, 'id']
    result.at[index, 'Title'] = search_df.at[index, 'title']
    result.at[index, 'Abstract'] = abst
    result.at[index, 'Claims'] = claims
    result.at[index, 'Inventors'] = search_df.at[index, 'inventor/author']
    result.at[index, 'Current Assignee'] = search_df.at[index, 'assignee']
    result.at[index, 'Publication Date'] = search_df.at[index, 'publication date']
    result.at[index, 'URL'] = search_df.at[index, 'result link']

    # 결과 데이터프레임과 스크랩되지 않은 링크 목록을 5번 반복할 때마다 저장합니다.
    #if i % 5 == 0:
    # result.to_csv(join(script_path, 'patents_data.csv'))
   #  with open(join(script_path, 'not_scrap_pickle'), 'wb') as fp:
   #     pickle.dump(not_scraped, fp)
    # Google 차단을 피하기 위해 10번 반복할 때마다 5초 동안 대기합니다.
    #if i % 10 == 0 and i != 0:
       # time.sleep(5)

# 결과 데이터프레임과 스크랩되지 않은 링크 목록을 최종 저장합니다.
result.to_csv(join(script_path, 'patents_data_0910.csv'))
with open(join(script_path, 'not_scrap_pickle_0910'), 'wb') as fp:
    pickle.dump(not_scraped, fp)


 99% (5175 of 5176) |################### | Elapsed Time: 0:35:11 ETA:   0:00:00